In [1]:
# Librerias, archivos y configuración

import warnings
import os
import pandas as pd
import funciones as f

warnings.filterwarnings('ignore')

In [2]:
# Se llama a la función que importa y realiza la limpieza para cada data frame

# Primero se crea una lista con los nombres de los archivos para poder llamar a la función

archivos = os.listdir('../../../../../../../../Volumes/JFreire/Databases/trafic_mad')
archivos.sort(reverse=True)

lista_meses = [e for e in archivos]

# Se llama a la función, la cual devuelve una lista de data frames

dfs = f.limpieza(lista_meses)

 98%|█████████▊| 96/98 [17:52<00:22, 11.17s/it]

14.00.csv


In [3]:
# Se concatenan todos los data frames en uno solo

df = pd.concat(dfs)

In [4]:
df.head()

,fecha,intensidad
0,2022-12-01 00:00:00,222.189757
1,2022-12-01 00:15:00,205.673980
2,2022-12-01 00:30:00,152.004489
3,2022-12-01 00:45:00,130.600806
4,2022-12-01 01:00:00,116.415027


In [14]:
# Se rellenan los valores nulos con la media de la columna para esa hora

df['fecha'] = pd.to_datetime(df['fecha'])

df['intensidad'] = df.groupby(df['fecha'].dt.hour)['intensidad'].transform(lambda x: x.fillna(x.mean()))

In [15]:
df.head()

,fecha,intensidad
0,2022-12-01 00:00:00,222.189757
1,2022-12-01 00:15:00,205.673980
2,2022-12-01 00:30:00,152.004489
3,2022-12-01 00:45:00,130.600806
4,2022-12-01 01:00:00,116.415027


In [16]:
# Se guarda el df en un csv

df.to_csv('../final_data/trafic_data.csv')